## Construindo um modelo transformer sem uso de framework

Um modelo Transformer consiste em várias partes principais:

1. Camada de Embedding: Transforma as palavras em vetores numéricos de tamanho fixo
2. Mecanismo de atenção: Permite que o modelo foque em diferentes partes da entrada
3. Camada Encoder e Decoder: Processam os dados sequencialmente
4. Camada Linear e Softmax: para predições finais

Hiperparâmetros iniciais

In [1]:
# Imports
import numpy as np

In [2]:
# Dimensao do modelo
dim_model = 64

# Comprimento da sequencia
seq_length = 10

# Tamanho do vocabulario
vocab_size = 100

### Camada Embedding
A função embedding é utilizada para converter entradas sequenciais em vetores densos de tamanho fixo. Esses vetores são conhecidos como embeddings e são uma parte fundamental, em especial dos modelos de PLN.

Esse embeddings são fundamentais para modelos de aprendizado profundo em PLN, pois fornecem uma representação rica e densa de palavras ou tokens, capturando informações contextuais e semânticas que são essenciais para tarefas como tradução automática, classificação de texto, entre outras.

In [3]:
# Define a função para criar uma matriz embedding
def embedding(input, vocab_size, dim_model):

    # Cria uma matriz de embeddings onde cada linha representa um token do vacabulario
    # A matriz é inicializada com valores aleatorios normalmente distribuidos
    embed = np.random.randn(vocab_size, dim_model)

    # Para cada indice de token no input, seleciona o embedding correspondente da matriz
    # Retorna um array de embeddings correspondentes a sequencia de entrada
    return np.array([embed[i] for i in input])

### Mecanismo de atenção

No Transformer, Q, K e V são derivados da mesma entrada em camadas de atenção do encoder, mas de entradas diferentes no decoder (Q vem da saída da camada anterior do decoder, enquanto K e V vêm da saída do encoder). O mecanismo de atenção calcula um conjunto de pontuações (usando o produto escalar entre Q e K, daí o nome "scaled dot-product attention"), aplica uma função softmax para obter pesos de atenção e usa esses pesos para ponderar os values, criando uma saída que é uma combinação ponderada das informações relevantes de entrada.

Este processo permite que o modelo dê "atenção" às partes mais relevantes da entrada para cada parte da saída, o que é especialmente útil em tarefas como tradução, onde a relevância de diferentes palavras da entrada pode variar dependendo da parte da frase que está sendo traduzida.

### Função de Ativação e Softmax

A função softmax é uma função de ativação amplamente utilizada em rede neurais, especialmente em cenários de classificação, onde é importante transformar valores brutos de saída (logits) em probabilidades que somam 1. Abaixo, está o código da função softmax.

In [4]:
# Função de ativação softmax
def softmax(x):

    # Calcula o exponencial de cada elemento do input, ajustado pelo máximo valor no input
    # para evitar overflow numérico
    e_x = np.exp(x - np.max(x))

    # Divide cada exponencial pelo somatório dos exponenciais ao longo do último eixo (axis = 1)
    # O reshape (-1,1) garante que a divisão seja realizada corretamente em um contexto multidimensional
    return e_x / e_x.sum(axis = 1).reshape(-1,1)

### Scale Dot Product

A função scaled_dot_product_attention() é um componente do mecanismo de atenção em modelos Transformer. Ela calcula a atenção entre conjuntos de queries (Q), keys (K) e values (V).

Essencialmente, esse função permite que o modelo dê importância diferenciada a diferentes partes da entrada, um aspecto chave que torna os modelos Transformer particularmente eficazes para tarefas de PLN e outras tarefas sequenciais.

In [5]:
# Define a função para calcular a atenção escalada por produto escalar
def scaled_dot_product_attention(Q, K, V):

    # Calcula o produto escalar entre Q e a transposta de K
    matmul_qk = np.dot(Q, K.T)

    # Obtém a dimensão dos vetores de chave
    depth = K.shape[-1]

    # Escala os logits dividindo-os pela raiz quadrada da profundidade
    logits = matmul_qk / np.sqrt(depth)

    # Aplica a função softmax para obter os pesos da atenção
    attention_weights = softmax(logits)

    # Multiplica os pesos de atenção pelos valores de V para obter a saída final
    output = np.dot(attention_weights, V)

    # Retorna a saída ponderada
    return output

### Saída do modelo com operação linear softmax

A função linear_and_softmax() é uma combinação de uma camada linear seguida por uma função softmax, comumente usada em modelos de aprendizado profundo, especialmente em tarefas de classificação

In [6]:
# Define a função que aplica uma transformação linear seguida de softmax
def linear_and_softmax(input):

    # Inicializa uma matriz de pesos com valores aleatórios normalmente distribuídos
    # Esta matriz conecta cada dimensão do modelo (dim model) a cada palavra do vocabulário (vocab_size)
    weights = np.random.randn(dim_model, vocab_size)

    # Realiza a operação linear (produto escalar) entre a entrada e a matriz de pesos
    # O resultado, logits, é um vetor que representa a entrada transformada em um espaço de maior dimensão
    logits = np.dot(input, weights)

    # Aplica a função softmax aos logits
    # Isso transforma os logits em um vetor de probabilidades, onde cada elemento soma 1
    return softmax(logits)

### Construindo o modelo final

In [7]:
# Função do modelo final
def transformer_model(input):

    # Embedding
    embedded_input = embedding(input, vocab_size, dim_model)

    # Mecanismo de atenção
    attention_output = scaled_dot_product_attention(embedded_input, embedded_input, embedded_input)

    # Camada linear e softmax
    output_probabilities = linear_and_softmax(attention_output)

    # Escolhendo os índices com maior probabilidade
    output_indices = np.argmax(output_probabilities, axis=1)

    return output_indices

### Usando o modelo para previsões

In [8]:
# Gerando dados aleatórios para a entrada do modelo
input_sequence = np.random.randint(0, vocab_size, seq_length)

In [9]:
print("Sequência de Entrada: ", input_sequence)

Sequência de Entrada:  [53 61 22 12 76 54 71 90 14 40]


In [10]:
# Fazendo previsões com o modelo
output = transformer_model(input_sequence)

In [11]:
print("Saída do modelo: ", output)

Saída do modelo:  [ 9 37 43 16 45 28 89 89 68 33]
